In [1]:
import argparse
import pathlib
import numpy as np
import cv2
from tensorboard.backend.event_processing import event_accumulator

In [2]:
#读取对应的tensorboard文件，将图片保存到指定的文件夹
args = argparse.Namespace(
        #val
        # log_dir='/data/bob/git/bob/resfusion_restore_test/lightning_logs/version_7/events.out.tfevents.1710918247.52d6188444b4.27829.1',
        # outdir='../images/val',
        #test
        log_dir='../../resfusion_results/ISTD/RDDM_Unet/events.out.tfevents.1712996585.ed673c9b596d.3037.1',
        outdir='../../resfusion_results/ISTD/RDDM_Unet/images/test',
        )

In [3]:
ea=event_accumulator.EventAccumulator(args.log_dir, size_guidance={'images': 0}) 
ea.Reload()
outdir = pathlib.Path(args.outdir)
outdir.mkdir(exist_ok=True, parents=True)
for tag in ea.Tags()['images']:
    events = ea.Images(tag)
    parts = tag.split("/")
    image_type = parts[0]
    image_code = parts[1]
    dirpath = outdir / image_type
    dirpath.mkdir(exist_ok=True, parents=True)

    for index, event in enumerate(events):
        s = np.frombuffer(event.encoded_image_string, dtype=np.uint8)
        if image_type == "Mask":
            image = cv2.imdecode(s, cv2.IMREAD_GRAYSCALE)
        else:
            image = cv2.imdecode(s, cv2.IMREAD_COLOR)
        outpath = dirpath / '{}.png'.format(image_code)
        cv2.imwrite(outpath.as_posix(), image)